In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import re
import collections

In [13]:
df_doc = pd.read_excel("https://github.com/tbwa-digital-strategy/Blue-Bird-Bio/blob/master/HCPStateLevelData.xlsx?raw=true")
df_med_cent = pd.read_excel("https://github.com/tbwa-digital-strategy/Blue-Bird-Bio/blob/master/medical%20centers.xlsx?raw=true")
df_pop = pd.read_csv("https://raw.githubusercontent.com/tbwa-digital-strategy/Blue-Bird-Bio/master/scprc-est2016-18%2Bpop-res.csv")

# print(list(df_med_cent['data'].values))
# use only one of the following lines, whichever you prefer
# for i in list(df_med_cent['data'].values):
#     print(re.findall("[(][\d]{3}[)][ ]?[\d]{3}-[\d]{4}", str(i)))
filtered = [i for i in list(df_med_cent['data'].values) if len(re.findall(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", str(i))) == 0]

revised_filtered_lst = []

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

for i, row in enumerate(filtered):
    if i % 4 == 2:
        revised_filtered_lst.append(str(row)[-2:])
    else:
        revised_filtered_lst.append(row)


        
bonemarrow_nums_by_state = {}        
for bucket in chunks(revised_filtered_lst, 4):
#     print(bucket)
    if bucket[2] in bonemarrow_nums_by_state:
        bonemarrow_nums_by_state[bucket[2]] += bucket[3]
    else:
        bonemarrow_nums_by_state[bucket[2]] = bucket[3]
    



for i, row_doc in df_doc.iterrows():
    for j, row_pop in df_pop.iterrows():
        if row_pop["NAME"].lower() in row_doc["State"].lower():
            df_doc.at[i, "Population"] = row_pop["POPESTIMATE2016"]
            
df_doc["Density Endos (per capita)"] = df_doc["Endos"]/df_doc["Population"]
df_doc["Density Opthamologists (per capita)"] = df_doc["Opthamologists"]/df_doc["Population"]
df_doc["Density Thalassemia (per capita)"] = df_doc["Thalassemia Patients"]/df_doc["Population"]
df_doc["Density Hematology Oncologists (per Thalassemia capita)"] = df_doc["Hematology Oncologists"]/df_doc["Thalassemia Patients"]
df_doc["Population Density"] = df_doc["Population"]/df_doc["Square Mileage"]

df_doc["code"] = df_doc["State"].apply(lambda x: x.split(",")[1])

for code in df_doc["code"]:
    if code not in bonemarrow_nums_by_state:
        bonemarrow_nums_by_state[code] = 0

df_doc.replace([np.inf, -np.inf], 0)
od = collections.OrderedDict(sorted(bonemarrow_nums_by_state.items()))
df_doc['Bone Marrow Medical Centers by State'] = list(od.values())
df_doc['Bone Marrow Medical Centers by Thalassemia Patients'] = df_doc['Bone Marrow Medical Centers by State']/df_doc["Thalassemia Patients"]
print(od)
print(df_doc["Density Hematology Oncologists (per Thalassemia capita)"])
print(df_doc.columns)

OrderedDict([('AK', 0), ('AL', 169), ('AR', 11), ('AZ', 341), ('CA', 1887), ('CO', 367), ('CT', 87), ('DC', 63), ('DE', 39), ('FL', 922), ('GA', 500), ('HI', 2), ('IA', 100), ('ID', 0), ('IL', 655), ('IN', 197), ('KS', 249), ('KY', 108), ('LA', 64), ('MA', 979), ('MD', 689), ('ME', 0), ('MI', 604), ('MN', 544), ('MO', 559), ('MS', 53), ('MT', 0), ('NC', 586), ('ND', 0), ('NE', 118), ('NH', 38), ('NJ', 363), ('NM', 0), ('NV', 0), ('NY', 1293), ('OH', 848), ('OK', 82), ('OR', 226), ('PA', 776), ('RI', 22), ('SC', 102), ('SD', 16), ('TN', 488), ('TX', 1615), ('UT', 220), ('VA', 191), ('VT', 0), ('WA', 569), ('WI', 380), ('WV', 64), ('WY', 0)])
Index(['State', 'Endos', 'Opthamologists', 'Rheumatolosits ',
       'Allergy & Immunology', 'Neurologists', 'Gastroenterologists ',
       'Infectious Disease', 'Population', 'Density Endos (per capita)',
       'Density Opthamologists (per capita)', 'code',
       'Rheumatoid Arthritis (%)', 'Diabetes Mellitus (%)', 'Square Mileage',
       'Adult

In [14]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

scl=[[0.0, 'rgb(165,0,38)'], [0.1111111111111111, 'rgb(215,48,39)'], [0.2222222222222222, 'rgb(244,109,67)'], [0.3333333333333333, 'rgb(253,174,97)'], [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)'], [0.6666666666666666, 'rgb(171,217,233)'], [0.7777777777777778, 'rgb(116,173,209)'], [0.8888888888888888, 'rgb(69,117,180)'], [1.0, 'rgb(49,54,149)']]

In [15]:
data8 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_doc['code'],
        z = df_doc["Density Thalassemia (per capita)"],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Thalassemia Patients By State")
        ) ]

layout2 = dict(
        title = 'Thalassemia Patients per State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig2 = dict( data=data8, layout=layout2 )
iplot( fig2, filename='thalassemia-cloropleth-map' )

In [18]:
data7 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_doc['code'],
        z = df_doc["Density Hematology Oncologists (per Thalassemia capita)"],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Density Hematology Oncologists (per Thalassemia capita)")
        ) ]

layout2 = dict(
        title = 'Density Hematology Oncologists (per Thalassemia capita)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig2 = dict( data=data7, layout=layout2 )
iplot( fig2, filename='hematology-cloropleth-map' )

In [19]:
data9 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_doc['code'],
        z = df_doc["Bone Marrow Medical Centers by Thalassemia Patients"],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Bone Marrow Medical Centers by Thalassemia Patients")
        ) ]

layout2 = dict(
        title = 'Bone Marrow Medical Centers by Thalassemia Patients',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig2 = dict( data=data9, layout=layout2 )
iplot( fig2, filename='bone-marrow-medical-centers-cloropleth-map' )